In [7]:
#Importing finviz API
from finvizfinance.quote import finvizfinance
#Importing neo4j drivers
#Graphdatabase to run driver
#Routing control to decide read/write operations
from neo4j import GraphDatabase, RoutingControl

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "V3rYstr0ngp@sSw0rd")

In [3]:
#Getting stock of Tesla into stock data item
stock = finvizfinance('tsla')
#Getting stock data into stock_fundament
stock_fundament = stock.ticker_fundament()
stock_fundament

{'Company': 'Tesla Inc',
 'Sector': 'Consumer Cyclical',
 'Industry': 'Auto Manufacturers',
 'Country': 'USA',
 'Exchange': 'NASD',
 'Index': 'NDX, S&P 500',
 'P/E': '185.01',
 'EPS (ttm)': '1.73',
 'Insider Own': '12.91%',
 'Shs Outstand': '3.22B',
 'Perf Week': '0.14%',
 'Market Cap': '1030.46B',
 'Forward P/E': '129.99',
 'EPS next Y': '2.46',
 'Insider Trans': '-0.26%',
 'Shs Float': '2.81B',
 'Perf Month': '7.28%',
 'Enterprise Value': '1007.01B',
 'PEG': '14.36',
 'EPS next Q': '0.49',
 'Inst Own': '48.84%',
 'Short Float': '2.49%',
 'Perf Quarter': '16.03%',
 'Income': '6.06B',
 'P/S': '11.11',
 'EPS this Y': '-29.45%',
 'Inst Trans': '1.21%',
 'Short Ratio': '0.66',
 'Perf Half Y': '-18.54%',
 'Sales': '92.72B',
 'P/B': '13.32',
 'EPS next Y Percentage': '43.95%',
 'ROA': '5.02%',
 'Short Interest': '69.96M',
 'Perf YTD': '-20.89%',
 'Book/sh': '23.98',
 'P/C': '27.60',
 'EPS next 5Y': '12.88%',
 'ROE': '8.43%',
 '52W High': '488.54 -34.61%',
 'Perf Year': '53.84%',
 'Cash/sh':

In [4]:
#Test query to return databases
with GraphDatabase.driver(auth=AUTH, uri=URI) as driver:
    result = driver.execute_query(
            "show databases yield name",
            database_="neo4j", routing_=RoutingControl.READ
            )

In [5]:
#Test code to check if specified database exists in database list
flag = False
for record in result.records:
    if record["name"] == "finviz":
        flag = True
    
if(flag):
    print("Found")
else:
    print("Not found")
    

Not found


In [6]:
#Query to create database
with GraphDatabase.driver(auth=AUTH, uri=URI) as driver:
    driver.execute_query(
        "create database experimental",
        database_="neo4j", routing_=RoutingControl.WRITE
    )

ClientError: {code: Neo.ClientError.Database.ExistingDatabaseFound} {message: Failed to create the specified database 'experimental': Database name or alias already exists.}

In [ ]:
#Formatting stock_fundament to input as query into neo4j
properties = ", ".join([f"`{key}` : $`{key}`" for key in stock_fundament.keys()])
#Creating query data item as string to pass into driver query
query = f"CREATE (:stock {{{properties}}})"

In [ ]:
#Adding stock data to database
with GraphDatabase.driver(uri=URI, auth=AUTH) as driver:
    driver.execute_query(query, stock_fundament, database_="experimental", routing_=RoutingControl.WRITE)

In [17]:
#Getting stock data into new_res to use as data
with GraphDatabase.driver(uri=URI, auth=AUTH) as driver:
    new_res, _, _ = driver.execute_query(
        "match (n: stock {Company : $Company})"
        "return n",
        Company = "Tesla",
        database_="experimental",
        routing_=RoutingControl.READ
    )

In [18]:
#Outputting the stock data
print(len(new_res))

0


In [ ]:
#Alternate method to get stock data
stock_fundament = finvizfinance('tsla').ticker_fundament()
stock_fundament

{'Company': 'Tesla Inc',
 'Sector': 'Consumer Cyclical',
 'Industry': 'Auto Manufacturers',
 'Country': 'USA',
 'Exchange': 'NASD',
 'Index': 'NDX, S&P 500',
 'P/E': '184.38',
 'EPS (ttm)': '1.73',
 'Insider Own': '12.91%',
 'Shs Outstand': '3.22B',
 'Perf Week': '-0.21%',
 'Market Cap': '1026.92B',
 'Forward P/E': '129.54',
 'EPS next Y': '2.46',
 'Insider Trans': '-0.26%',
 'Shs Float': '2.81B',
 'Perf Month': '6.91%',
 'Enterprise Value': '1003.47B',
 'PEG': '14.31',
 'EPS next Q': '0.49',
 'Inst Own': '48.84%',
 'Short Float': '2.49%',
 'Perf Quarter': '15.63%',
 'Income': '6.06B',
 'P/S': '11.08',
 'EPS this Y': '-29.45%',
 'Inst Trans': '1.21%',
 'Short Ratio': '0.66',
 'Perf Half Y': '-18.82%',
 'Sales': '92.72B',
 'P/B': '13.28',
 'EPS next Y Percentage': '43.95%',
 'ROA': '5.02%',
 'Short Interest': '69.96M',
 'Perf YTD': '-21.16%',
 'Book/sh': '23.98',
 'P/C': '27.50',
 'EPS next 5Y': '12.88%',
 'ROE': '8.43%',
 '52W High': '488.54 -34.83%',
 'Perf Year': '53.31%',
 'Cash/sh'

In [30]:
#Function to add ticker to node given company name
def addTicker(company_name, ticker):
    with GraphDatabase.driver(auth=AUTH, uri=URI) as driver:
        driver.execute_query(
        "MATCH (n:stock {Company : $company})"
        "SET n.Ticker = $ticker",
        company = company_name,
        ticker = ticker,
        database_="fin-proj",
        routing_=RoutingControl.WRITE
    )

In [21]:
f = open("Stock tickers.txt", "r")

names = []

for x in f:
    stock_data = finvizfinance(x[:-1]).ticker_fundament()
    names.append(stock_data["Company"])

In [ ]:
f = open("Stock tickers.txt", "r")

i = 0

for x in f:
    string = str(x[:-1])
    addTicker(names[i], string)
    i += 1
    
f.close()

KeyboardInterrupt: 

: 